In [ ]:
import os
from pathlib import Path
import numpy as np
from tqdm.auto import tqdm
import pickle
from collections import defaultdict
import importlib
from matplotlib import pyplot as plt
from tqdm.auto import tqdm

from themachinethatgoesping.echosounders import kongsbergall, simradraw, index_functions
from themachinethatgoesping import pingprocessing as pp
from themachinethatgoesping import tools as ptools

In [ ]:
import themachinethatgoesping as tmtgp
tmtgp.version()

In [ ]:
#create the output path
path_out = "../unittest_data/"
path_in = "./data_in/"

os.makedirs(path_out,exist_ok=True)
os.makedirs(path_in,exist_ok=True)

## Explore and sort input data

In [ ]:
folders = index_functions.find_folders_with_files(path_in, [".all",".wcd", '.raw'], followlinks=True)
folders.sort()
N = 10

for input_path in folders:
    print(input_path)

In [ ]:
all_files = index_functions.find_files(folders, ['.raw','.wcd','.all'], followlinks=True, verbose=False)
prg = tqdm(total=len(all_files), desc="Creating test data", unit="files")

for input_path in folders:
    for endings, prefix, postfix, InputFileHandler in [
        [
            ".raw",
            "raw",
            ".raw",
            simradraw.SimradRawFileHandler,
        ],
        [
            ".all",
            "all",
            ".all",
            kongsbergall.KongsbergAllFileHandler,
        ],
        [
            ".wcd",
            "wcd",
            ".wcd",
            kongsbergall.KongsbergAllFileHandler,
        ],
    ]:

        prg.set_postfix_str(f"{input_path} / {prefix}")

        prg.set_description(f"finding files")
        input_files = index_functions.find_files(input_path, endings, followlinks=True, verbose=False)

        if not input_files:
            print(f"- No {endings} files in {input_path}")
            continue

        for file in input_files:
            try:
                prg.set_postfix_str(f"{file}")

                # open and index files
                prg.set_description(f"opening files")

                fm = InputFileHandler(file, show_progress=False)

                if(len(fm.get_pings()) < N):
                    raise RuntimeError(f"Less then {N} pings in file")

                # create new file name
                file_name, file_extension = os.path.splitext(os.path.split(file)[1])
                
                prg.set_description(f"creating new file name")
                output_file = os.path.join(
                    input_path.replace(path_in, path_out), str(hash(file_name)) + file_extension
                )  # + hash of the old file name
                # print(output_file)
                os.makedirs(os.path.split(output_file)[0], exist_ok=True)

                prg.set_postfix_str(f"Writing file {output_file}")

                #get timestamps of all datagrams in Nth ping
                timestamps = [d.get_timestamp() for d in fm.get_pings()[N].file_data.datagrams()]

                # append timestamp of second PositionDatagram and second AttitudeDatagram
                if prefix in ["all", "wcd"]:                    
                    if len(fm.datagram_interface.datagrams("PositionDatagram")) > 1:
                        timestamps.append(fm.datagram_interface.datagrams("PositionDatagram")[1].get_timestamp())    
                    if len(fm.datagram_interface.datagrams("AttitudeDatagram")) > 1:                
                        timestamps.append(fm.datagram_interface.datagrams("AttitudeDatagram")[1].get_timestamp())
                else:
                    timestamps.append(fm.datagram_interface.datagrams("NME0")[1].get_timestamp())

                #compute largest timestamp
                max_timestamp = np.nanmax(timestamps)

                with open(output_file, "wb") as ofi:
                    for d in fm.datagram_interface.datagrams():
                        if d.get_timestamp() > max_timestamp:
                            break
                        ofi.write(d.to_binary())
            except Exception as e:
                print(f"--- Error with {file} ---")
                print(f"Error: {e}")
                print("---")
            prg.update(1)


In [ ]:
input_files

In [ ]:
fm = simradraw.SimradRawFileHandler(input_files[4], show_progress=False)

In [ ]:
print(fm)